In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

def add_basic_stats(row, moves, pref=""):

    row[pref + 'winrate_mean'] = np.mean(moves.winrate_delta)
    row[pref + 'score_mean'] = np.mean(moves.score_delta)
    row[pref + 'score_var'] = np.var(moves.score_delta)
    row[pref + 'selfplay_mean'] = np.mean(moves.selfplay_delta)
    row[pref + 'utility_mean'] = np.mean(moves.utility_delta)
    row[pref + 'winrate_beauty_percent'] = np.mean([x > 0 for x in moves.winrate_delta])
    row[pref + 'score_beauty_percent'] = np.mean([x > 0 for x in moves.score_delta])


def add_advanced_stats(row, moves, pref=""):
    moves.score_delta.sort()
    moves.winrate_delta.sort()

    row[pref + 'score25p'] = moves.score_delta[int(moves.cnt_moves * 0.25)]
    row[pref + 'score75p'] = moves.score_delta[int(moves.cnt_moves * 0.75)]
    row[pref + 'score_max'] = np.max(moves.score_delta)
    row[pref + 'score_min'] = np.min(moves.score_delta)
    row[pref + 'winrate25p'] = moves.winrate_delta[int(moves.cnt_moves * 0.25)]
    row[pref + 'winrate75p'] = moves.winrate_delta[int(moves.cnt_moves * 0.75)]

    row[pref + 'score_five_best_mean'] = np.mean(moves.score_delta[-5:])
    row[pref + 'score_five_worst_mean'] = np.mean(moves.score_delta[:5])

    row[pref + 'stddev_last'] = moves.stddev_delta[-1]
    moves.stddev_delta.sort()
    row[pref + 'stddev_mean'] = np.mean(moves.stddev_delta)
    row[pref + 'stddev50p'] = moves.stddev_delta[int(moves.cnt_moves * 0.5)]

    row[pref + 'score50p'] = moves.score_delta[int(moves.cnt_moves * 0.5)]

    row[pref + 'winrate_midmean'] = np.mean(
        moves.winrate_delta[int(moves.cnt_moves * 0.25):int(moves.cnt_moves * 0.75)])
    row[pref + 'score_midmean'] = np.mean(moves.score_delta[int(moves.cnt_moves * 0.25):int(moves.cnt_moves * 0.75)])


def get_index_dan(rank):
  for i in range(len(rank)):
    if rank[i] in ['k', 'd']:
      return i
  return -1


def get_int_from_rank(rank):
    ind = get_index_dan(rank)
    if ind < 0 or rank[0] == 'P' or not rank[:ind].isdigit():
        return None
    if rank[ind] == 'k':
        return -int(rank[:ind]) + 1
    else:
        return int(rank[:ind])


def get_rank_from_int(x):
    if x > 0:
        return str(x) + "d"
    else:
        return str(-x + 1) + "k"


def int_from_player(player):
    return int(player == 'W')


def player_from_int(x):
    return 'W' if x == 1 else 'B'


def add_meta(row, player='W'):
    if row['Result'] == '?':
        row['game_result'] = 0
    else:
        row['game_result'] = int(row['Result'])
        if player == 'B':
            row['game_result'] = -row['game_result']
    row['color'] = int_from_player(player)
    row['rank'] = get_int_from_rank(row[player + '_rating'])
    row['game_length'] = len(row['W_move']) + len(row['B_move'])


def convert_to_lists(df):
    for i, row in tqdm(df.iterrows()):
        row['kek'] = i
        row['W_scoreLead'] = [float(x) for x in row['W_scoreLead'].split()]
        row['B_scoreLead'] = [float(x) for x in row['B_scoreLead'].split()]
        row['W_scoreSelfplay'] = [float(x) for x in row['W_scoreSelfplay'].split()]
        row['B_scoreSelfplay'] = [float(x) for x in row['B_scoreSelfplay'].split()]
        row['W_scoreStdev'] = [float(x) for x in row['W_scoreStdev'].split()]
        row['B_scoreStdev'] = [float(x) for x in row['B_scoreStdev'].split()]
        row['W_utility'] = [float(x) for x in row['W_utility'].split()]
        row['B_utility'] = [float(x) for x in row['B_utility'].split()]
        row['W_winrate'] = [float(x) for x in row['W_winrate'].split()]
        row['B_winrate'] = [float(x) for x in row['B_winrate'].split()]
        df.loc[i] = row

class MovesInfo:
    def __init__(self, row, n_moves=None, player='W'):
        enemy = 'B' if player == 'W' else 'W'
        if player == 'B':
            moves_len = min(len(row['B_winrate']), len(row['W_winrate']) + 1)
        else:
            moves_len = min(len(row['W_winrate']), len(row['B_winrate']))

        if n_moves is None:
            start_ind = 0
        else:
            start_ind = max(moves_len - n_moves - 1, 0)

        end_ind = moves_len
        w_0 = []
        sc_0 = []
        ut_0 = []
        std_0 = []
        sf_0 = []
        if player == 'B':
            if start_ind == 0:
                w_0 = [0.475]
                sc_0 = [0.]
                ut_0 = [0.]
                std_0 = [19. - 0.07666535852999867 * 2]
                sf_0 = [0.]
            else:
                w_0 = [row[enemy + '_winrate'][start_ind - 1]]

                sc_0 = [row[enemy + '_scoreLead'][start_ind - 1]]
                ut_0 = [row[enemy + '_utility'][start_ind - 1]]
                std_0 = [row[enemy + '_scoreStdev'][start_ind - 1]]
                sf_0 = [row[enemy + '_scoreSelfplay'][start_ind - 1]]
        self.winrate_delta = np.array(row[player + '_winrate'][start_ind:end_ind]) - np.array(
            (w_0 + row[enemy + '_winrate'])[start_ind:end_ind])
        self.score_delta = np.array(row[player + '_scoreLead'][start_ind:end_ind]) - np.array(
            (sc_0 + row[enemy + '_scoreLead'])[start_ind:end_ind])
        self.utility_delta = np.array(row[player + '_utility'][start_ind:end_ind]) - np.array(
            (ut_0 + row[enemy + '_utility'])[start_ind:end_ind])
        self.selfplay_delta = np.array(row[player + '_scoreSelfplay'][start_ind:end_ind]) - np.array(
            (sf_0 + row[enemy + '_scoreSelfplay'])[start_ind:end_ind])
        self.stddev_delta = np.array(row[player + '_scoreStdev'][start_ind:end_ind]) - np.array(
            (std_0 + row[enemy + '_scoreStdev'])[start_ind:end_ind])

        if player == 'B':
            self.winrate_delta = -self.winrate_delta
            self.score_delta = -self.score_delta
            self.utility_delta = -self.utility_delta
            self.selfplay_delta = -self.selfplay_delta

        self.move = row[player + '_move'].split()
        self.cnt_moves = end_ind - start_ind


def add_all_game_stats(df, player = 'W'):
    df['winrate_mean'] = None
    df['score_midmean'] = None
    df['score_mean'] = None
    df['score_var'] = None
    df['winrate_beauty_percent'] = None
    df['score_beauty_percent'] = None
    df['utility_mean'] = None
    df['score25p'] = None
    df['score75p'] = None
    df['winrate25p'] = None
    df['winrate75p'] = None
    df['winrate_midmean'] = None
    df['score50p'] = None
    df['game_length'] = None
    df['rank'] = None
    df['color'] = None
    df['game_result'] = None
    df['selfplay_mean'] = None
    df['stddev_mean'] = None
    df['stddev50p'] = None
    df['stddev_last'] = None
    df['score_max'] = None
    df['score_min'] = None
    df['score_five_best_mean'] = None
    df['score_five_worst_mean'] = None

    for i, row in tqdm(df.iterrows()):
        add_meta(row, player=player)
        add_basic_stats(row, MovesInfo(row, player=player))
        add_advanced_stats(row, MovesInfo(row, player=player))
        df.loc[i] = row


def get_start_of_yose(margin_moves, no_change_count=5):
    '''
    Находим сколько последних ходов мы будем считать что это йосе

    Будем идти с конца по массиву количества ходов на краю, если в массиве no_change_count одинаковых чисел,
    то мы останавливаемся и говорим, что тут началось йосе
    '''
    no_change = 1
    ans = len(margin_moves)
    for i in range(len(margin_moves) - 2, -1, -1):
        if margin_moves[i] == margin_moves[i + 1]:
            no_change += 1
        else:
            no_change = 1
        if no_change >= no_change_count:
            ans = len(margin_moves) - i - 1
            break
    return ans


def reset_basic_stats(df, pref):
    df[pref + 'winrate_mean'] = None
    df[pref + 'winrate_beauty_percent'] = None
    df[pref + 'score_beauty_percent'] = None
    df[pref + 'score_mean'] = None
    df[pref + 'selfplay_mean'] = None
    df[pref + 'score_var'] = None
    df[pref + 'utility_mean'] = None


def is_marginal_move(move):
    return (move[0] == 'a' or move[0] == 'b') or (move[0] == 'r' or move[0] == 's') or \
           (move[1] == 'a' or move[1] == 'b') or (move[1] == 'r' or move[1] == 's')


def count_of_marginal_moves(moves):
    ans = np.zeros(len(moves))
    for i in range(len(moves)):
        ans[i] = is_marginal_move(moves[i])
    ans = np.cumsum(ans)
    return ans


def add_yose_stats(df, player = 'W'):
    pref = 'yose_'
    reset_basic_stats(df, pref)
    df['yose_length'] = None
    df['yose_start'] = None
    df['yose_has'] = None
    for i, row in tqdm(df.iterrows()):
        marginal_moves = count_of_marginal_moves(row['W_move'].split())
        n_moves = get_start_of_yose(marginal_moves, 10)
        add_basic_stats(row, MovesInfo(row, n_moves, player=player), pref)
        row['yose_length'] = n_moves
        row['yose_start'] = len(row['W_move'].split()) - n_moves
        row['yose_has'] = row['yose_start'] != 0
        df.loc[i] = row


def delta_moves(a, b):
    return abs(ord(a[0]) - ord(b[0])) + abs(ord(a[1]) - ord(b[1]))


def get_distance_of_moves(moves):
    ans = np.zeros(len(moves) - 1)
    for i in range(1, len(moves)):
        ans[i - 1] = delta_moves(moves[i], moves[i - 1])
    return ans


def get_distance_from_enemy(my_moves, enemy_moves):
    ans = np.zeros(min(len(my_moves), len(enemy_moves)))
    for i in range(min(len(my_moves), len(enemy_moves))):
        ans[i] = delta_moves(my_moves[i], enemy_moves[i])
    return ans


def add_last_moves_stats(df, n_moves, pref=None, player='W'):
    if pref is None:
        pref = str(n_moves) + "_"
    reset_basic_stats(df, pref)
    for i, row in tqdm(df.iterrows()):
        add_basic_stats(row, MovesInfo(row, n_moves, player=player), pref)
        df.loc[i] = row


def add_dist_stats_to_row(row, player='W'):
    enemy = 'W' if player == 'B' else 'W'
    dist = get_distance_of_moves(row[player+'_move'].split())
    dist_enemy = get_distance_from_enemy(row[player + '_move'].split(), row[enemy + '_move'].split())
    dist.sort()

    row['dist_mean'] = np.mean(dist)
    row['dist_var'] = np.var(dist)
    row['dist_median'] = dist[len(dist) // 2]
    row['dist_percent_more_than_10'] = np.mean([x > 10 for x in dist])
    row['dist_percent_more_than_5'] = np.mean([x > 5 for x in dist])
    row['dist_percent_more_than_20'] = np.mean([x > 20 for x in dist])

    row['dist_from_enemy_mean'] = np.mean(dist_enemy)
    row['dist_from_enemy_var'] = np.var(dist_enemy)


def add_dist_stats(df, player='W'):
    df['dist_mean'] = None
    df['dist_var'] = None
    df['dist_median'] = None
    df['dist_percent_more_than_5'] = None
    df['dist_percent_more_than_10'] = None
    df['dist_percent_more_than_20'] = None

    df['dist_from_enemy_mean'] = None
    df['dist_from_enemy_var'] = None

    for i, row in tqdm(df.iterrows()):
       if len(row['W_move']) > 20:
        add_dist_stats_to_row(row, player=player)
        df.loc[i] = row


def delete_non_scalar_parameters(df):
    df.drop(['W_rating', 'B_rating', 'W_move', 'B_move', 'W_scoreLead', 'B_scoreLead', 'W_scoreSelfplay',
             'B_scoreSelfplay', 'W_scoreStdev', 'B_scoreStdev', 'W_utility', 'B_utility',
             'W_winrate', 'B_winrate', 'Result'], axis=1, inplace=True)


def add_delta_lists_to_row(row, moves, player = 'W'):
    row['winrate'] = moves.winrate_delta
    row['score'] = moves.score_delta
    row['winrate_sqr'] = np.array([x ** 2 for x  in moves.winrate_delta])
    row['score_sqr'] = np.array([x ** 2 for x  in moves.score_delta])
    row['utility'] = moves.utility_delta
    row['selfplay'] = moves.selfplay_delta
    row['stddev'] = moves.stddev_delta
    row['dist_from_prev'] = get_distance_of_moves(moves.move)
    row['dist_more_5'] = [int(x > 5) + 1 for x in row['dist_from_prev']]
    enemy = 'B' if player == 'W' else 'W'
    row['dist_enemy'] = get_distance_from_enemy(row[player + '_move'].split(), row[enemy + '_move'].split())


def add_lists_to_df(df, player='W'):
    df['winrate'] = None
    df['score'] = None
    df['utility'] = None
    df['selfplay'] = None
    df['stddev'] = None
    df['dist_from_prev'] = None
    df['rank'] = None
    df['color'] = None
    df['game_length'] = None
    df['score_sqr'] = None
    df['winrate_sqr'] = None
    df['dist_more_5'] = None
    df['dist_enemy'] = None
    for i, row in tqdm(df.iterrows()):
        add_meta(row, player)
        add_delta_lists_to_row(row, MovesInfo(row, player=player))
        df.loc[i] = row


def get_feature_df(df, player='W'):
    convert_to_lists(df)
    add_all_game_stats(df, player)
    add_yose_stats(df, player)
    add_last_moves_stats(df, 10, player)
    add_last_moves_stats(df, 20, player)
    add_dist_stats(df, player)
    delete_non_scalar_parameters(df)
    return df

def add_meta_to_df(df, player='W'):
    df['rank'] = None
    df['color'] = None
    df['game_length'] = None
    for i, row in tqdm(df.iterrows()):
        try:
            add_meta(row, player)
            df.loc[i] = row
        except Exception as e:
            print(e)
            df = df.drop(index=[i])

def get_df_with_lists(df, player='W'):
    add_meta_to_df(df, player)
    df = df[df['game_length'] >= 40]
    convert_to_lists(df)
    add_all_game_stats(df, player)
    #add_last_moves_stats(df, 20, player = player)
    #add_dist_stats(df, player)
    add_lists_to_df(df, player)
    #delete_non_scalar_parameters(df)
    return df


In [2]:
from google.colab import drive
drive.mount('/content/drive')
katago = pd.read_csv('/content/drive/MyDrive/GO_dataset_after_script/15k-9d/data.csv')[-80000:]
katago.dropna(how = 'any', inplace= True)

Mounted at /content/drive


In [3]:
katago.dropna(how = 'any', inplace= True)

In [4]:
katago_white = get_df_with_lists(katago, player = 'W')

79658it [01:49, 729.29it/s]
0it [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)
79615it [02:23, 556.38it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:156: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See t

In [5]:
katago_all = katago_white
katago_all.dropna(how='any', inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [6]:
features = ['score', 'winrate', 'utility', 'stddev', 'selfplay', 'dist_from_prev', 
            'W_winrate', 'B_winrate', 'W_scoreLead', 'B_scoreLead', 'W_scoreSelfplay',
             'B_scoreSelfplay', 'W_scoreStdev', 'B_scoreStdev', 'dist_enemy']

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(katago_all[features], katago_all['rank'], test_size=0.4)

In [8]:
bins = np.array([-8.5, -2.5, 3.5, 10.0])
y_train_big = np.digitize(y_train, bins)

first_model_train_idx = y_train.index[y_train <= -8]
second_model_train_idx = y_train.index[(y_train >= -9) & (y_train <= -2)]
third_model_train_idx = y_train.index[(y_train >= -3) & (y_train <= 4)]
fourth_model_train_idx = y_train.index[(y_train >= 3) & (y_train <= 9)]

X_train_first = X_train.loc[first_model_train_idx]
X_train_second = X_train.loc[second_model_train_idx]
X_train_third = X_train.loc[third_model_train_idx]
X_train_fourth = X_train.loc[fourth_model_train_idx]

y_train_first = y_train.loc[first_model_train_idx]
y_train_second = y_train.loc[second_model_train_idx]
y_train_third = y_train.loc[third_model_train_idx]
y_train_fourth = y_train.loc[fourth_model_train_idx]

In [9]:
second_model_train_idx

Int64Index([154594, 190510, 216506, 200209, 151749, 188778, 159649, 207354,
            172978, 202075,
            ...
            181927, 204203, 202004, 216266, 179547, 147176, 209526, 216664,
            207857, 176860],
           dtype='int64', length=16629)

In [10]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers

def get_tensor(df, features, sequence_len):
    def resize_to_length(m, length):
        if len(m) > length:
            return m[:length]
        n_nules = length - len(m)
        return list([0] * n_nules) + list(m)

    arr = []
    for feature in features:
        arr.append([np.array(y) for y in df[feature].apply(lambda x: resize_to_length(x, sequence_len))])
    arr = np.array(arr)
    arr = arr.swapaxes(0, 1)
    arr = arr.swapaxes(1, 2)
    return tf.convert_to_tensor(arr, np.float32)


class RnnKerasRunner:
    __DEFAULT_FEATURES = ['score', 'winrate', 'utility', 'stddev', 'selfplay', 'dist_from_prev', 'winrate_sqr', 'score_sqr', 'dist_more_5']

    def __create_rnn(self, hidden_units, input_shape):
        model = keras.Sequential()
        # model.add(layers.Embedding(input_dim=6, output_dim=6))
        model.add(tf.keras.layers.Masking(mask_value=0.,
                                          input_shape=input_shape))
        model.add(layers.BatchNormalization())
        model.add(layers.Dense(units=12, activation="linear"))
        model.add(layers.Dense(32, activation='relu'))
        model.add(layers.Bidirectional(layers.LSTM(hidden_units)))  
        model.add(layers.LeakyReLU())
        model.add(layers.Dense(units=1, activation="linear"))
        model.compile(loss='mean_squared_error', optimizer='adam', metrics="mean_absolute_error")
        return model

    def __init__(self, model = None, sequence_len=150, features=__DEFAULT_FEATURES):
        self.features = features
        self.sequence_len = sequence_len
        if model is None:
            self.model = self.__create_rnn(128, (sequence_len, len(features)))
        else:
            self.model = model

    def fit(self, df, target, epochs=10, batch_size=256):
        X = get_tensor(df, self.features, self.sequence_len)
        self.model.fit(X, tf.convert_to_tensor(target, np.float32), epochs=epochs,
                       batch_size=batch_size)

    def predict(self, df):
        X = get_tensor(df, self.features, self.sequence_len)
        return np.array([y[0] for y in self.model.predict(X)])

In [11]:
model_big = RnnKerasRunner(sequence_len=150, features = features)
model_first = RnnKerasRunner(sequence_len=150, features = features)  
model_second = RnnKerasRunner(sequence_len=150, features = features)  
model_third = RnnKerasRunner(sequence_len=150, features = features)  
model_fourth = RnnKerasRunner(sequence_len=150, features = features)  

In [12]:
model_big.fit(X_train, y_train_big, epochs=5, batch_size = 256)
model_first.fit(X_train_first, y_train_first, epochs=5, batch_size = 256)
model_second.fit(X_train_second, y_train_second, epochs=5, batch_size = 256)
model_third.fit(X_train_third, y_train_third, epochs=5, batch_size = 256)
model_fourth.fit(X_train_fourth, y_train_fourth, epochs=5, batch_size = 256)

Epoch 1/5
187/187 [==============================] - 186s 931ms/step - loss: 0.7332 - mean_absolute_error: 0.6864
Epoch 2/5
187/187 [==============================] - 173s 925ms/step - loss: 0.4779 - mean_absolute_error: 0.5423
Epoch 3/5
187/187 [==============================] - 173s 925ms/step - loss: 0.4615 - mean_absolute_error: 0.5290
Epoch 4/5
187/187 [==============================] - 173s 925ms/step - loss: 0.4480 - mean_absolute_error: 0.5196
Epoch 5/5
187/187 [==============================] - 172s 920ms/step - loss: 0.4368 - mean_absolute_error: 0.5123
Epoch 1/5
52/52 [==============================] - 59s 935ms/step - loss: 23.6536 - mean_absolute_error: 3.6017
Epoch 2/5
52/52 [==============================] - 48s 930ms/step - loss: 4.0907 - mean_absolute_error: 1.7580
Epoch 3/5
52/52 [==============================] - 48s 923ms/step - loss: 4.0713 - mean_absolute_error: 1.7529
Epoch 4/5
52/52 [==============================] - 48s 925ms/step - loss: 4.0654 - mean_absolute

In [13]:
y_pred_big = np.round(model_big.predict(X_test))

In [14]:
X_test = X_test.reset_index(drop=True)

In [15]:
test_idx_first = np.where(y_pred_big == 0)[0]
test_idx_second = np.where(y_pred_big == 1)[0]
test_idx_third = np.where(y_pred_big == 2)[0]
test_idx_fourth = np.where(y_pred_big == 3)[0]

X_test_first = X_test.loc[test_idx_first]
X_test_second = X_test.loc[test_idx_second]
X_test_third = X_test.loc[test_idx_third]
X_test_fourth = X_test.loc[test_idx_fourth]

y_test_first = np.take(y_test, test_idx_first)
y_test_second = np.take(y_test, test_idx_second)
y_test_third = np.take(y_test, test_idx_third)
y_test_fourth = np.take(y_test, test_idx_fourth)

In [16]:
y_pred_first = model_first.predict(X_test_first)
y_pred_second = model_second.predict(X_test_second)
y_pred_third = model_third.predict(X_test_third)
y_pred_fourth = model_fourth.predict(X_test_fourth)

In [17]:
first_sum_error = np.sum(np.abs(y_test_first - y_pred_first)) 
second_sum_error = np.sum(np.abs(y_test_second - y_pred_second)) 
third_sum_error = np.sum(np.abs(y_test_third - y_pred_third)) 
fourth_sum_error = np.sum(np.abs(y_test_fourth - y_pred_fourth))

first_sum_error, second_sum_error, third_sum_error, fourth_sum_error

(7662.260513305664, 43098.3878660202, 32383.17721181363, 18408.76809644699)

In [18]:
mae = (first_sum_error + second_sum_error + third_sum_error + fourth_sum_error) / len(y_test)
mae

3.1888649653829835

In [19]:
model_big.model.save('/content/drive/MyDrive/rnn_big.h5')
model_first.model.save('/content/drive/MyDrive/rnn_first.h5')
model_second.model.save('/content/drive/MyDrive/rnn_second.h5')
model_third.model.save('/content/drive/MyDrive/rnn_third.h5')
model_fourth.model.save('/content/drive/MyDrive/rnn_fourth.h5')